In [14]:
import numpy
import pandas as pd

In [15]:
df = pd.read_csv("veri-seti.csv",header=0,infer_datetime_format=True)
df.shape

(498, 10)

In [16]:
df.head()

,0,1,22.01,101,3.08,6.42,1008.45,0.1,268.6,0.0
0,1,1,22.06,105,2.94,6.28,1008.24,0,269.1,0.0
1,2,1,22.06,106,2.82,6.08,1007.90,0,269.7,0.0
2,3,1,21.87,106,2.63,5.74,1007.61,0,270.4,0.0
3,4,1,21.53,100,2.39,5.30,1007.51,0,271.3,0.0
4,5,1,21.56,94,2.09,4.47,1008.03,0,271.6,0.0


In [17]:
df = df.iloc[:,1:10]
df.head()

,1,22.01,101,3.08,6.42,1008.45,0.1,268.6,0.0
0,1,22.06,105,2.94,6.28,1008.24,0,269.1,0.0
1,1,22.06,106,2.82,6.08,1007.90,0,269.7,0.0
2,1,21.87,106,2.63,5.74,1007.61,0,270.4,0.0
3,1,21.53,100,2.39,5.30,1007.51,0,271.3,0.0
4,1,21.56,94,2.09,4.47,1008.03,0,271.6,0.0


In [18]:
df.tail()

,1,22.01,101,3.08,6.42,1008.45,0.1,268.6,0.0
493,0,6.13,319,3.77,4.50,1011.58,5,274.5,0.0
494,0,5.74,321,4.08,5.20,1010.65,3,274.7,0.0
495,0,6.05,323,3.94,5.68,1010.31,1,274.9,0.0
496,0,6.85,326,3.61,6.08,1010.73,0,275.7,0.0
497,0,7.65,325,3.12,5.62,1011.00,0,276.4,0.0


In [19]:
dataset=numpy.array(df)
dataset

array([[  1.  ,  22.06, 105.  , ...,   0.  , 269.1 ,   0.  ],
       [  1.  ,  22.06, 106.  , ...,   0.  , 269.7 ,   0.  ],
       [  1.  ,  21.87, 106.  , ...,   0.  , 270.4 ,   0.  ],
       ...,
       [  0.  ,   6.05, 323.  , ...,   1.  , 274.9 ,   0.  ],
       [  0.  ,   6.85, 326.  , ...,   0.  , 275.7 ,   0.  ],
       [  0.  ,   7.65, 325.  , ...,   0.  , 276.4 ,   0.  ]])

# 1- Parti Boyutunu ve Dönem Sayısı Nasıl Ayarlanır

In [20]:
# Epochs ve batch size'ı ızgarada aramak için scikit-learn'i kullanalım.
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [21]:
# KerasClassifier için gerekli olan modeli oluşturma 
def create_model():
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,1:9]
Y = dataset[:,0:1]
# model oluşturma
model = KerasClassifier(build_fn=create_model, verbose=0)
# Izgara arama parametrelerini tanımladık.
batch_size = [2, 4, 8, 16, 32, 64, 128]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.809237 using {'batch_size': 8, 'epochs': 100}
0.678715 (0.098537) with: {'batch_size': 2, 'epochs': 10}
0.716867 (0.073451) with: {'batch_size': 2, 'epochs': 50}
0.730924 (0.092979) with: {'batch_size': 2, 'epochs': 100}
0.636546 (0.059092) with: {'batch_size': 4, 'epochs': 10}
0.700803 (0.027090) with: {'batch_size': 4, 'epochs': 50}
0.795181 (0.044268) with: {'batch_size': 4, 'epochs': 100}
0.640562 (0.118593) with: {'batch_size': 8, 'epochs': 10}
0.753012 (0.055865) with: {'batch_size': 8, 'epochs': 50}
0.809237 (0.032003) with: {'batch_size': 8, 'epochs': 100}
0.604418 (0.124854) with: {'batch_size': 16, 'epochs': 10}
0.765060 (0.027386) with: {'batch_size': 16, 'epochs': 50}
0.781124 (0.070310) with: {'batch_size': 16, 'epochs': 100}
0.608434 (0.078698) with: {'batch_size': 32, 'epochs': 10}
0.763052 (0.035582) with: {'batch_size': 32, 'epochs': 50}
0.748996 (0.034896) with: {'batch_size': 32, 'epochs': 100}
0.536145 (0.114405) with: {'batch_size': 64, 'epochs': 10}
0.4518

# 2- Eğitim Optimizasyon Algoritmasını Ayarlama

In [22]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [23]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(optimizer='adam'):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,1:9]
Y = dataset[:,0:1]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.801205 using {'optimizer': 'Adam'}
0.403614 (0.078081) with: {'optimizer': 'SGD'}
0.682731 (0.053956) with: {'optimizer': 'RMSprop'}
0.556225 (0.158036) with: {'optimizer': 'Adagrad'}
0.556225 (0.110569) with: {'optimizer': 'Adadelta'}
0.801205 (0.025558) with: {'optimizer': 'Adam'}
0.755020 (0.032750) with: {'optimizer': 'Adamax'}
0.726908 (0.098537) with: {'optimizer': 'Nadam'}


# 3- Öğrenme Hızını ve Momentum Ayarlama

In [24]:
# Öğrenme hızını ve momentumu ızgarada aramak için scikit-learn'i kullanalım
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD

In [25]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(learn_rate=0.01, momentum=0):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Modeli derleme
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,1:9]
Y = dataset[:,0:1]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
learn_rate = [0.001, 0.005, 0.0015, 0.01, 0.025]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.620482 using {'learn_rate': 0.001, 'momentum': 0.2}
0.495984 (0.143346) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.620482 (0.121183) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.403614 (0.078081) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.403614 (0.078081) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.403614 (0.078081) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.403614 (0.078081) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.403614 (0.078081) with: {'learn_rate': 0.005, 'momentum': 0.0}
0.403614 (0.078081) with: {'learn_rate': 0.005, 'momentum': 0.2}
0.403614 (0.078081) with: {'learn_rate': 0.005, 'momentum': 0.4}
0.403614 (0.078081) with: {'learn_rate': 0.005, 'momentum': 0.6}
0.403614 (0.078081) with: {'learn_rate': 0.005, 'momentum': 0.8}
0.403614 (0.078081) with: {'learn_rate': 0.005, 'momentum': 0.9}
0.487952 (0.137722) with: {'learn_rate': 0.0015, 'momentum': 0.0}
0.403614 (0.078081) with: {'learn_rate': 0.0015, 'momentum': 0.2}
0.403614 (0.078081) with: {'

# 4- Ağ Ağırlığı Başlatma Nasıl Ayarlanır

In [26]:
# Ağırlık başlatmayı ızgarada aramak için scikit-learn'i kullanalım.
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [27]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(init_mode='uniform'):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer=init_mode, activation='relu'))
	model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,1:9]
Y = dataset[:,0:1]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.815261 using {'init_mode': 'uniform'}
0.815261 (0.015027) with: {'init_mode': 'uniform'}
0.751004 (0.075294) with: {'init_mode': 'lecun_uniform'}
0.765060 (0.013014) with: {'init_mode': 'normal'}
0.403614 (0.078081) with: {'init_mode': 'zero'}
0.740964 (0.061631) with: {'init_mode': 'glorot_normal'}
0.761044 (0.065500) with: {'init_mode': 'glorot_uniform'}
0.763052 (0.078903) with: {'init_mode': 'he_normal'}
0.763052 (0.046749) with: {'init_mode': 'he_uniform'}


# 5- Nöron Aktivasyon Fonksiyonu Nasıl Ayarlanır

In [28]:
# Aktivasyon işlevini ızgarada aramak için scikit-learn'i kullanın
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [29]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(activation='relu'):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation=activation))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,1:9]
Y = dataset[:,0:1]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.819277 using {'activation': 'linear'}
0.447791 (0.128011) with: {'activation': 'softmax'}
0.785141 (0.012378) with: {'activation': 'softplus'}
0.702811 (0.062281) with: {'activation': 'softsign'}
0.773092 (0.024263) with: {'activation': 'relu'}
0.403614 (0.078081) with: {'activation': 'tanh'}
0.700803 (0.066417) with: {'activation': 'sigmoid'}
0.403614 (0.078081) with: {'activation': 'hard_sigmoid'}
0.819277 (0.027386) with: {'activation': 'linear'}


# 6- Bırakma Düzenlemesini Ayarlama

In [30]:
# Bırakma oranını ızgarada aramak için scikit-learn'i kullanalım.
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

In [31]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(dropout_rate=0.0, weight_constraint=0):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(weight_constraint)))
	model.add(Dropout(dropout_rate))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,1:9]
Y = dataset[:,0:1]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
#Izgara arama parametrelerini tanımladık.
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.817269 using {'dropout_rate': 0.2, 'weight_constraint': 4}
0.791165 (0.019878) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.755020 (0.015811) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.815261 (0.040060) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.791165 (0.037567) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.712851 (0.086088) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.759036 (0.069734) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.811245 (0.007513) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.803213 (0.039452) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.799197 (0.044631) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.799197 (0.023245) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.805221 (0.027090) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.807229 (0.040261) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.791165 (0.061500) with: {'dropout_rate': 0.2, 'weight_constraint': 

# 7- Gizli Katmandaki Nöron Sayısı Nasıl Ayarlanır

In [32]:
# Nöron sayısını ızgarada aramak için scikit-learn'i kullanalım.
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

In [33]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(neurons=1):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(neurons, input_dim=8, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(4)))
	model.add(Dropout(0.2))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,1:9]
Y = dataset[:,0:1]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.811245 using {'neurons': 15}
0.795181 (0.047434) with: {'neurons': 1}
0.797189 (0.032750) with: {'neurons': 5}
0.789157 (0.054772) with: {'neurons': 10}
0.811245 (0.038205) with: {'neurons': 15}
0.781124 (0.018622) with: {'neurons': 20}
0.795181 (0.026027) with: {'neurons': 25}
0.803213 (0.034547) with: {'neurons': 30}
